In [1]:
# P1 — écrire requirements.txt depuis le venv actif
from pathlib import Path, sys
import os
ROOT = Path.cwd() if (Path.cwd()/ "data" / "processed").exists() else Path.cwd().parent
req = ROOT/"requirements.txt"
os.system(f"{sys.executable} -m pip freeze > {req}")
print("Écrit :", req)


Écrit : c:\Users\FFix\Desktop\Formation Data Science\Projects\OpenClassroom\Project 07\projet7_starter\requirements.txt


In [2]:
# P2 — crée un .gitignore simple
from pathlib import Path
ROOT = Path.cwd() if (Path.cwd()/ "data" / "processed").exists() else Path.cwd().parent
gi = ROOT/".gitignore"
gi.write_text("""# Python
__pycache__/
*.pyc
.ipynb_checkpoints/
.venv/

# Données volumineuses
data/raw/
data/processed/*.csv

# Modèles / figures
models/*.joblib
reports/figures/
""", encoding="utf-8")
print("Écrit :", gi)


Écrit : c:\Users\FFix\Desktop\Formation Data Science\Projects\OpenClassroom\Project 07\projet7_starter\.gitignore


In [3]:
# P3 — écrire src/predict_cli.py
from pathlib import Path
ROOT = Path.cwd() if (Path.cwd()/ "data" / "processed").exists() else Path.cwd().parent
SRC = ROOT/"src"; SRC.mkdir(exist_ok=True)
code = r'''
import argparse, json
from pathlib import Path
import pandas as pd, numpy as np, joblib

def main():
    parser = argparse.ArgumentParser(description="Prédire avec le modèle RF baseline")
    parser.add_argument("--input", required=True, help="CSV featuré (colonnes = features du modèle)")
    parser.add_argument("--output", required=True, help="CSV de sortie (avec proba et prédiction)")
    args = parser.parse_args()

    ROOT = Path(__file__).resolve().parents[1]
    MODELS = ROOT/"models"
    meta = json.load(open(MODELS/"model_meta.json"))
    feat = meta["features"]; thr = meta["threshold"]

    rf  = joblib.load(MODELS/"rf_model.joblib")
    imp = joblib.load(MODELS/"imputer.joblib")

    df = pd.read_csv(args.input)
    # sécurité: créer colonnes manquantes à NaN, et garder l'ordre
    for c in feat:
        if c not in df.columns:
            df[c] = np.nan
    X = df[feat].astype(float).values
    X = imp.transform(X)
    proba = rf.predict_proba(X)[:,1]
    pred  = (proba >= thr).astype(int)

    out = df.copy()
    out["proba"] = proba
    out["pred"]  = pred
    Path(args.output).parent.mkdir(parents=True, exist_ok=True)
    out.to_csv(args.output, index=False)
    print("Écrit:", args.output, "| lignes:", len(out))

if __name__ == "__main__":
    main()
'''
(SRC/"predict_cli.py").write_text(code, encoding="utf-8")
print("Écrit :", SRC/"predict_cli.py")


Écrit : c:\Users\FFix\Desktop\Formation Data Science\Projects\OpenClassroom\Project 07\projet7_starter\src\predict_cli.py
